# PDWT Model #

Hanjo's algo

First we set up the environment.

In [1]:
%matplotlib inline

import numpy as np
import copy
import math
import os
import glob
import matplotlib.pyplot as plt
from scipy.io.wavfile import read
import IPython.display as ipd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive

INT16_FAC = (2**15)-1
INT32_FAC = (2**31)-1
INT64_FAC = (2**63)-1
norm_fact = {'int16':INT16_FAC, 'int32':INT32_FAC, 'int64':INT64_FAC,'float32':1.0,'float64':1.0}

global iF  # The input file name
global xR  # The raw input samples 
global x   # The input samples normalized
global fs  # The input sample rate

This is the heap state of the algorithm. This can be considered the workspace of the algorithm, and is not yet optimized 

In [2]:
kxmx_pdwt_f_state = {
    'sample_rate': 0,
    'current_index': 0,
    'block_size': 0,
    'spectrum_size': 0,
    'golden_f0': 0.0,
    'lowest_frequency': 0.0,
    'highest_frequency': 0.0,
    'frequencies': [],
    'fourier_coef_r': [],
    'fourier_coef_i': [],
    'spectrum_r': [],
    'spectrum_i': [],
    'spectrum_magnitude': [],
    'spectrum_phase': [],
    'last_spectrum_r': [],
    'last_spectrum_i': [],
    'golden_samples': [],
    'samples_in': [],
    'samples_out': []
}

This is the state initialization function.  This allocates and populates the memory required by the state struct, and initializes some values, including the Fourier coefficients. 

This is intended to be called only once, making performance optimization less important

In [3]:
def kxmx_pdwt_f_init(sample_rate, block_size, spectrum_size, lowest_frequency, highest_frequency, golden_samples, golden_f0):
    state = copy.deepcopy(kxmx_pdwt_f_state)
    
    # Start out by initializing our state parameters
    state['current_index'] = 0
    state['sample_rate'] = sample_rate
    state['block_size'] = block_size
    state['spectrum_size'] = spectrum_size
    state['lowest_frequency'] = lowest_frequency
    state['highest_frequency'] = highest_frequency
    state['golden_f0'] = golden_f0

    state['golden_samples'] = golden_samples

    # Allocate the memory needed for the sample buffers
    state['samples_in'] = np.zeros(block_size, np.float32)
    state['samples_out'] = np.zeros(block_size, np.float32)

    # Allocate the memory needed for the spectrums
    state['spectrum_r'] = np.zeros(spectrum_size, np.float32)
    state['spectrum_i'] = np.zeros(spectrum_size, np.float32)
    state['spectrum_magnitude'] = np.zeros(spectrum_size, np.float32)
    state['spectrum_phase'] = np.zeros(spectrum_size, np.float32)
    state['last_spectrum_r'] = np.zeros(spectrum_size, np.float32)
    state['last_spectrum_i'] = np.zeros(spectrum_size, np.float32)

    # Allocate the memory needed for the frequencies
    state['frequencies'] = np.zeros(spectrum_size, np.float32)

    # Calculate the frequencies
    for i in range(0, spectrum_size):
        state['frequencies'][i] = i * \
            ((highest_frequency - lowest_frequency) /
             float(spectrum_size)) + lowest_frequency

    # Allocate the memory needed for the fourier coefficients
    state['fourier_coef_r'] = np.zeros(spectrum_size, np.float32)
    state['fourier_coef_i'] = np.zeros(spectrum_size, np.float32)

    # Calculate the fourier coefficients
    for i in range(0, spectrum_size):
        state['fourier_coef_r'][i] = math.sin(
            -2 * math.pi * state['frequencies'][i] / spectrum_size)
        state['fourier_coef_i'][i] = math.cos(
            -2 * math.pi * state['frequencies'][i] / spectrum_size)

    # Return the intialized state object
    return state

This is the tight loop method, called once per sample (Block operation is not yet implemented).  Here is where performance is most important.

In [4]:
def kxmx_pdwt_f_update(sample, state):
    S = sample
    N = state['spectrum_size']
    R = state['spectrum_r']
    I = state['spectrum_i']
    mX = state['spectrum_magnitude']
    i = state['current_index']
    x = state['samples_in']
    old_S = x[i]
    x[i] = S
    S_delta = S - old_S

    for j in range(0, N):
        coef_i = (j * i) % N
        coef_r = state['fourier_coef_r'][coef_i]
        coef_i = state['fourier_coef_i'][coef_i]
        R[j] = R[j] + coef_r * S_delta
        I[j] = I[j] + coef_i * S_delta
        mX[j] = math.sqrt(math.pow(R[j], 2) + math.pow(I[j], 2))

    i += 1
    if i >= state['block_size']:
        i = 0
    
    state['current_index'] = i


This is the analysis method, which calles the update function every sample, and collects the magnitude spectrum for plotting.

In [5]:
def pdwt_analysis(x, state):
    xmX = []                                       # Initialise empty list for mX
    for i in range(0, x.size):
        kxmx_pdwt_f_update(x[i], state)
        xmX.append(np.array(state['spectrum_magnitude'])) # Append output to list
    xmX = np.array(xmX)                             
    return xmX
    

This is the system method, the parameters are handed off by the UI widgets below, allowing the above methods to be run with different parameters.  The resulting magnitude spectrum is pl

In [6]:
def pdwt_system(p_N, p_fFrom, p_fTo):
    global N, M, H, wN, w, mX, pX, y, yR
    
    N = p_N
    fFrom = p_fFrom
    fTo = p_fTo
    
    state = kxmx_pdwt_f_init(fs, N, N, fFrom, fTo, 0, 440.0)

    mX = pdwt_analysis(x, state)
    
     # create figure to plot
    plt.figure(figsize=(17, 10))
    
     # frequency range to plot
    maxplotfreq = fTo
    
    # plot magnitude spectrogram
    numFrames = int(mX[:,0].size)
    frmTime = np.arange(numFrames)/float(fs)
    binFreq = fs*np.arange(N*maxplotfreq/fs)/N
    plt.pcolormesh(frmTime, binFreq, np.transpose(mX[:,:int(N*maxplotfreq/fs+1)]))
    plt.xlabel('time (sec)')
    plt.ylabel('frequency (Hz)')
    plt.title('magnitude spectrogram')
    plt.autoscale(tight=True)

# Playground

Here you can play with a few different inputs, change some parameters

In [7]:
def read_input_file(p_iF):
    global iF, fs, xR, x
    iF = p_iF
    # Read the input file now
    fs, xR = read(iF)
    x = np.float32(xR)/norm_fact[xR.dtype.name]
    display(ipd.Audio(xR, rate=fs))
    
files = glob.glob('audio/*.wav')
interact(read_input_file, p_iF = widgets.Dropdown(options=files,description='Audio File:'))
interact_manual(pdwt_system, 
                p_N=widgets.IntSlider(value=128,min=8,max=1024,step=1,description='Num. Freqs.'),
                p_fFrom=widgets.FloatSlider(value=100.0,min=10.0,max=5000.0,step=1,description='Freq. From'),
                p_fTo=widgets.FloatSlider(value=1000.0,min=10.0,max=5000.0,step=1,description='Freq. To')) 

interactive(children=(Dropdown(description='Audio File:', options=('audio/sine-440.wav', 'audio/speech-male.wav', 'audio/singing-female.wav', 'audio/sine-440-490.wav', 'audio/cello-phrase.wav', 'audio/oboe-A4.wav', 'audio/sax-phrase-short.wav'), value='audio/sine-440.wav'), Output()), _dom_classes=('widget-interact',))

interactive(children=(IntSlider(value=128, description='Num. Freqs.', max=1024, min=8), FloatSlider(value=100.0, description='Freq. From', max=5000.0, min=10.0, step=1.0), FloatSlider(value=1000.0, description='Freq. To', max=5000.0, min=10.0, step=1.0), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

<function __main__.pdwt_system>